# Visualization - Relative Timings App

Relative timings between different methods.

A plot inspired by one in the COBRAS paper.

## Preliminaries

### Imports

In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import json
import sys
import pickle as pkl
import warnings

from os.path import dirname

# Dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State

In [2]:
# Custom

root_dir = dirname(dirname(os.getcwd()))
src_dir = os.path.join(root_dir, 'src')
sys.path.append(src_dir)

import exp
from exp.utils.extra import mem_usage
from exp.runner.RunExp import RunExp
from exp.runner.RunMercs import RunMercs

from exp.visual.menus import (generate_dropdown_menu,
                              generate_dropdown_menus_from_df,
                              generate_slider_menu)
from exp.visual.plots import (generate_graph)
from exp.visual.callback import (extract_menu_inputs_menu_names_from_layout,
                                filter_dataframe)

In [3]:
from exp.eval.relative_time_plot import *
from exp.eval.preprocess import *

In [4]:
root_dir

'/cw/dtailocal/Dropbox/Files/KUL/research/codebases/homework'

### Methods

Some custom methods I need in this notebook.

In [5]:
def merge_aggregated_outputs_multiple_exps(exp_idxs, **kwargs):
    """
    Merge aggregated outputs from multiple experiments.
    """
    
    f = collect_aggregated_outputs_from_exp
    
    gen = (f(exp_idx, **kwargs) for exp_idx in exp_idxs)
    
    result = {}
    for g in gen:
        result = {k: pd.concat([result.get(k, None),v], sort=False)
                  for k,v in g.items()}    
        
    return result

def collect_aggregated_outputs_from_exp(exp_idx, **kwargs):
    """
    Load the aggregated outputs by a single experiment.
    """
    
    # Preliminaries
    dfs = {}
    
    # Actions
    re = RunExp.load(idx=exp_idx, **kwargs)
    for output in re.aggr_outputs:
        dfs[output] = re.load_output(kind=output)
    return dfs

## Global Parameters

This is the single most important thing you need to specify, i.e., from which experiments do you want to collect the results?

In [6]:
exp_idxs = [10,13,15,17]

## Collect Data

Now, the actual work starts.

In [7]:
dfs = merge_aggregated_outputs_multiple_exps(exp_idxs, root_dir=root_dir)

In [8]:
dfs.keys()

dict_keys(['mod_config', 'qry_codes', 'timings', 'results'])

In [9]:
df_res = preprocess_aggr_df(dfs['results'], kind='res')
df_qry = preprocess_aggr_df(dfs['qry_codes'], kind='qry')
df_cfg = preprocess_aggr_df(dfs['mod_config'], kind='cfg')
df_tmg = preprocess_aggr_df(dfs['timings'], kind='tmg')

In [10]:
#df_cfg.head()
#df_res.head()
#df_tmg.head()
#df_qry.head()

In [11]:
df_cfg.head()

,dataset,predict.algo,mod.type,mod.keyword,predict.param,predict.its,fit.sel.its,fit.sel.param,fit.ind.max_depth,fit.ind.type,name
idx,,,,,,,,,,,
674,nltcs,MI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,predict.algo=MI|mod.type=Mercs|mod.keyword=md1...
675,nltcs,MAFI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,predict.algo=MAFI|mod.type=Mercs|mod.keyword=m...
676,msnbc,MI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,predict.algo=MI|mod.type=Mercs|mod.keyword=md1...
677,msnbc,MAFI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,predict.algo=MAFI|mod.type=Mercs|mod.keyword=m...
678,jester,MI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,predict.algo=MI|mod.type=Mercs|mod.keyword=md1...


In [12]:
df_lpt = build_relative_time_df(df_tmg,
                                df_cfg,
                                baseline_filter=[('mod.keyword', 'md16'), ('predict.algo', 'MI')],
                                baseline_column='inf_time',
                                group_on=('predict.algo', 'predict.its', 'predict.param'))

In [13]:
df_lpt.head()

,,ind_time,inf_time,dataset,predict.algo,mod.type,mod.keyword,predict.param,predict.its,fit.sel.its,fit.sel.param,fit.ind.max_depth,fit.ind.type,baseline_inf_time,relative_inf_time
idx,name,,,,,,,,,,,,,,
674,predict.algo=MI|predict.param=0.95|predict.its=0.1,1.707582,0.022731,nltcs,MI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,0.022731,1.000000
675,predict.algo=MAFI|predict.param=0.95|predict.its=0.1,1.707582,0.018550,nltcs,MAFI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,0.022731,0.816075
676,predict.algo=MI|predict.param=0.95|predict.its=0.1,72.326585,0.131463,msnbc,MI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,0.131463,1.000000
677,predict.algo=MAFI|predict.param=0.95|predict.its=0.1,72.326585,0.099202,msnbc,MAFI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,0.131463,0.754601
678,predict.algo=MI|predict.param=0.95|predict.its=0.1,45.259171,0.083209,jester,MI,Mercs,md16,0.95,0.1,4.0,2.0,16.0,DT,0.083209,1.000000


## Layout

In [14]:
graph_style = {'width':             '85%',
               'float':             'right',
               'z-index':            1,
               'position':           'relative',
               'margin-bottom':      '2cm'}

dropdown_menu_style = {'width':            '14%',
                       'backgroundColor':  'rgb(250, 250, 250)',
                       'float':            'left',
                       'z-index':            0,
                       'position':          'relative',
                       'border':            '1px solid gray'}

slider_menu_style = {'width':             '95%',
                     'backgroundColor':  'rgb(250, 250, 250)',
                     'float':            'left',
                     'margin-left':       '1cm',
                     'margin-bottom':     '2cm'}

extra_style = {'border':            '1px solid black'}

sep = html.Div(style={'clear': 'both'})

txt_box_style = {'width':             '25%',
                 'float':             'left',
                 'z-index':            1,
                 'position':           'relative',
                 'margin-left':       '2cm',
                 'border':            '1px solid gray'}

## Menus

### Dropdown

In [15]:
# Dropdown Menus
perf_dd_menus = generate_dropdown_menus_from_df(df_cfg, ignore_columns=['name'])

perf_dd_menus = html.Div(perf_dd_menus,
                         style=dropdown_menu_style)

/cw/dtailocal/Dropbox/Files/KUL/research/codebases/homework/src/exp/visual/menus.py:84: UserWarning:


            Could not sort this column. Typically because there is a mix
            of int/float and strings.
            



## Static App

In [16]:
#df_lpt.head()

In [17]:
scatterplot_fig = generate_graph(df_lpt,
                                 kind='scatter',
                                 x_field='dataset',
                                 y_field='relative_inf_time')

perf_graph = dcc.Graph(id='scatterplot', 
                       figure=scatterplot_fig)

stat_perf_contents = html.Div([perf_graph],
                              style=graph_style)

## Dynamic App

Re-uses some stuff from the static configuration.

### Initialization

In [18]:
dyn_perf_contents = [perf_dd_menus,
                     stat_perf_contents]

In [19]:
# Init App.
app = dash.Dash()
app.layout = html.Div(dyn_perf_contents)

In [20]:
menu_inputs, menu_names = extract_menu_inputs_menu_names_from_layout(app.layout)
#menu_names

### Main Callback

This callback method needs to handle everything at once.

In [ ]:
@app.callback(
    Output('scatterplot', 'figure'),
    menu_inputs)
def update_lineplot(*args):
    
    menus = zip(menu_names, args)
    
    df_filt = df_lpt
    
    for name, values in menus:
        print(name)
        df_filt = filter_dataframe(df_filt, name, values)


    scatterplot_fig = generate_graph(df_filt,
                                     kind='scatter',
                                     x_field='dataset',
                                     y_field='relative_inf_time')
    
    return scatterplot_fig

## Run App

Run the actual browser applet.

In [ ]:
app.run_server(port=8886)

 * Running on http://127.0.0.1:8886/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Feb/2019 16:52:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Feb/2019 16:52:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Feb/2019 16:52:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Feb/2019 16:52:49] "POST /_dash-update-component HTTP/1.1" 200 -


dataset
predict.algo
mod.type
mod.keyword
predict.param
predict.its
fit.sel.its
fit.sel.param
fit.ind.max_depth
fit.ind.type


/cw/dtailocal/Dropbox/Files/KUL/research/codebases/homework/src/exp/visual/plots.py:199: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

127.0.0.1 - - [12/Feb/2019 16:52:58] "POST /_dash-update-component HTTP/1.1" 200 -


dataset
predict.algo
mod.type
mod.keyword
predict.param
predict.its
fit.sel.its
fit.sel.param
fit.ind.max_depth
fit.ind.type


127.0.0.1 - - [12/Feb/2019 16:52:59] "POST /_dash-update-component HTTP/1.1" 200 -


...

dataset
predict.algo
mod.type
mod.keyword
predict.param
predict.its
fit.sel.its
fit.sel.param
fit.ind.max_depth
fit.ind.type


127.0.0.1 - - [12/Feb/2019 17:00:07] "POST /_dash-update-component HTTP/1.1" 200 -


dataset
predict.algo
mod.type
mod.keyword
predict.param
predict.its
fit.sel.its
fit.sel.param
fit.ind.max_depth
fit.ind.type


127.0.0.1 - - [12/Feb/2019 17:00:09] "POST /_dash-update-component HTTP/1.1" 200 -


dataset
predict.algo
mod.type
mod.keyword
predict.param
predict.its
fit.sel.its
fit.sel.param
fit.ind.max_depth
fit.ind.type
